In [1]:
import torch
torch.cuda.is_available()
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Using cuda device


In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[5.7069e-01, 8.1246e-01, 5.0527e-01],
        [8.1015e-01, 9.2638e-02, 1.2260e-01],
        [5.8904e-01, 1.7190e-01, 6.5483e-01],
        [6.8125e-01, 7.5629e-01, 8.0779e-01],
        [2.8977e-01, 4.0582e-01, 7.1985e-04]])


In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:08<00:00, 3.21MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 317kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.50MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.6MB/s]


In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297418  [   64/60000]
loss: 2.289218  [ 6464/60000]
loss: 2.273252  [12864/60000]
loss: 2.265676  [19264/60000]
loss: 2.243602  [25664/60000]
loss: 2.215650  [32064/60000]
loss: 2.219226  [38464/60000]
loss: 2.189528  [44864/60000]
loss: 2.172812  [51264/60000]
loss: 2.155484  [57664/60000]
Test Error: 
 Accuracy: 48.7%, Avg loss: 2.144941 

Epoch 2
-------------------------------
loss: 2.156307  [   64/60000]
loss: 2.155186  [ 6464/60000]
loss: 2.092710  [12864/60000]
loss: 2.107005  [19264/60000]
loss: 2.069326  [25664/60000]
loss: 1.996738  [32064/60000]
loss: 2.022110  [38464/60000]
loss: 1.947596  [44864/60000]
loss: 1.934324  [51264/60000]
loss: 1.887661  [57664/60000]
Test Error: 
 Accuracy: 58.1%, Avg loss: 1.877914 

Epoch 3
-------------------------------
loss: 1.917025  [   64/60000]
loss: 1.896901  [ 6464/60000]
loss: 1.767422  [12864/60000]
loss: 1.804723  [19264/60000]
loss: 1.727168  [25664/60000]
loss: 1.657836  [32064/600

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
